In [5]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException,NoSuchElementException,ElementNotInteractableException
import pandas as pd
import re
import undetected_chromedriver as uc
import numpy as np
import os
from datetime import datetime
import random
from time import sleep
import time, random

### Samsung: https://www.lazada.vn/dien-thoai-di-dong/samsung/?page={i}&q=Samsung%20Galaxy%20s23
### Iphone: https://www.lazada.vn/dien-thoai-di-dong/lazada6916025_apple-127197227/?page={i}&q=Iphone%2015

## Lấy links product

In [13]:
options = uc.ChromeOptions()
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)

df_list = []  # List to store data from each page

for i in range(1, 2):  # Corrected range for 2 pages
    driver.get(f"https://www.lazada.vn/dien-thoai-di-dong/lazada6916025_apple-127197227/?page={i}&q=Iphone%2015")
    time.sleep(random.randint(5, 10))  # Corrected function call

    elems = driver.find_elements(By.CSS_SELECTOR, ".RfADt [href]")
    titles = [elem.text for elem in elems]
    links = [elem.get_attribute('href') for elem in elems]
    elems_price = driver.find_elements(By.CSS_SELECTOR, ".aBrP0")
    prices = [elem.text for elem in elems_price]

    discount_percent_list, sold_quantity_list, count_rating_list = [], [], []

    # Corrected loop to iterate over links since title length and link length should match
    for j in range(1,len(titles)+1):
        # Adjusted indexing and variable usage
        try:
            discount_percent = driver.find_element(By.XPATH, f"/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[3]/div[{j}]/div/div/div[2]/div[4]/span").text
        except NoSuchElementException:
            discount_percent = None
        discount_percent_list.append(discount_percent)

        try:
            sold_quantity = driver.find_element(By.XPATH, f"/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[3]/div[{j}]/div/div/div[2]/div[5]/span[1]/span[1]").text
        except NoSuchElementException:
            sold_quantity = None
        sold_quantity_list.append(sold_quantity)

        try:
            count_rating = driver.find_element(By.XPATH, f"/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[3]/div[{j}]/div/div/div[2]/div[5]/div/span").text
        except NoSuchElementException:
            count_rating = None
        count_rating_list.append(count_rating)

    df = pd.DataFrame(list(zip(titles, prices, links, discount_percent_list, sold_quantity_list, count_rating_list)), columns=['title', 'price', 'link_item', 'discount_percent', 'sold_quantity', 'count_rating'])
    df['index_'] = np.arange(1, len(df) + 1)
    
    df_list.append(df)

final_df = pd.concat(df_list, ignore_index=True)

driver.quit()

print(final_df.head())


                                      title         price  \
0          iPhone 15 - Hàng Chính Hãng VN/A  20.290.000 ₫   
1     iPhone 15 Plus - Hàng Chính Hãng VN/A  23.190.000 ₫   
2  iPhone 15 Pro Max - Hàng Chính Hãng VN/A  31.590.000 ₫   
3          iPhone 11 - Hàng Chính Hãng VN/A   9.850.000 ₫   
4          iPhone 13 - Hàng Chính Hãng VN/A  14.790.000 ₫   

                                           link_item discount_percent  \
0  https://www.lazada.vn/products/iphone-15-hang-...          12% Off   
1  https://www.lazada.vn/products/iphone-15-plus-...          11% Off   
2  https://www.lazada.vn/products/iphone-15-pro-m...          10% Off   
3  https://www.lazada.vn/products/iphone-11-hang-...          55% Off   
4  https://www.lazada.vn/products/iphone-13-hang-...          41% Off   

  sold_quantity count_rating  index_  
0    717 Đã bán        (150)       1  
1    468 Đã bán         (92)       2  
2   8.4K Đã bán       (1875)       3  
3   9.0K Đã bán       (1536)       4  

In [20]:
# Lọc cho Samsung
def convert_sold_quantity(s):
    # Loại bỏ từ "Đã bán"
    number_part = s.split()[0]
    # Kiểm tra và chuyển đổi
    if 'K' in number_part:
        # Chuyển đổi từ "K" thành 1000
        return int(float(number_part.replace('K', '')) * 1000)
    else:
        # Chỉ trả về số nếu không có "K"
        return int(number_part)

def filter_samsung_galaxy_s23(df):
    # Sử dụng apply để chuyển đổi sold_quantity
    df['sold_quantity'] = df['sold_quantity'].apply(convert_sold_quantity)
    
    # Lọc DataFrame
    filtered_df = df[(df['title'].str.contains("Samsung Galaxy S23")) & (df['sold_quantity'] > 50)]
    
    return filtered_df


filtered_df = filter_samsung_galaxy_s23(final_df)
filtered_df.to_csv('product_links_Smarphone.csv', mode='a', header=not os.path.exists('product_links_Smarphone.csv'), index=False, encoding='utf-8-sig')

AttributeError: 'int' object has no attribute 'split'

In [19]:
final_df

,title,price,link_item,discount_percent,sold_quantity,count_rating,index_
0,iPhone 15 - Hàng Chính Hãng VN/A,20.290.000 ₫,https://www.lazada.vn/products/iphone-15-hang-...,12% Off,717,(150),1
1,iPhone 15 Plus - Hàng Chính Hãng VN/A,23.190.000 ₫,https://www.lazada.vn/products/iphone-15-plus-...,11% Off,468,(92),2
2,iPhone 15 Pro Max - Hàng Chính Hãng VN/A,31.590.000 ₫,https://www.lazada.vn/products/iphone-15-pro-m...,10% Off,8400,(1875),3
3,iPhone 11 - Hàng Chính Hãng VN/A,9.850.000 ₫,https://www.lazada.vn/products/iphone-11-hang-...,55% Off,9000,(1536),4
4,iPhone 13 - Hàng Chính Hãng VN/A,14.790.000 ₫,https://www.lazada.vn/products/iphone-13-hang-...,41% Off,4600,(970),5
5,iPhone 15 Pro - Hàng Chính Hãng VN/A,26.590.000 ₫,https://www.lazada.vn/products/iphone-15-pro-h...,8% Off,1500,(278),6
6,iPhone 14 - Hàng Chính Hãng VN/A,17.590.000 ₫,https://www.lazada.vn/products/iphone-14-hang-...,30% Off,908,(117),7
7,iPhone 12 - Hàng Chính Hãng VN/A,12.290.000 ₫,https://www.lazada.vn/products/iphone-12-hang-...,51% Off,3900,(809),8


In [18]:
# Lọc cho Iphone
def convert_sold_quantity(s):
    # Loại bỏ từ "Đã bán"
    number_part = s.split()[0]
    # Kiểm tra và chuyển đổi
    if 'K' in number_part:
        # Chuyển đổi từ "K" thành 1000
        return int(float(number_part.replace('K', '')) * 1000)
    else:
        # Chỉ trả về số nếu không có "K"
        return int(number_part)

def filter_iphone_15(df):
    """Filters DataFrame for iPhone 15 listings with sold_quantity > 50."""
    # Apply conversion to sold_quantity
    df['sold_quantity'] = df['sold_quantity'].apply(convert_sold_quantity)
    
    # Filter DataFrame based on title containing 'iPhone 15' and sold_quantity > 50
    filtered_df = df[(df['title'].str.contains("iPhone 15", case=False)) & (df['sold_quantity'] > 50)]
    
    return filtered_df

filtered_df = filter_iphone_15(final_df)
filtered_df.to_csv('product_links_Smarphone.csv', mode='a', header=not os.path.exists('product_links_Smarphone.csv'), index=False, encoding='utf-8-sig')

## Lấy Info Product

In [21]:
df = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang TMĐT\Lazada\product_links_Smarphone.csv")
links = df['link_item'].to_list()
links

['https://www.lazada.vn/products/iphone-15-hang-chinh-hang-vna-i2417259469.html',
 'https://www.lazada.vn/products/iphone-15-plus-hang-chinh-hang-vna-i2417254495.html',
 'https://www.lazada.vn/products/iphone-15-pro-max-hang-chinh-hang-vna-i2417233670.html',
 'https://www.lazada.vn/products/iphone-15-pro-hang-chinh-hang-vna-i2417249565.html']

In [16]:
def get_product_info(driver, link):
    products = []  # Danh sách để lưu trữ các link sản phẩm
    url = link
    driver.get(url)
    sleep(4.765)  # Điều chỉnh dựa trên thời gian thực tế cần để trang tải
    print(url)
    try:

        product_title = safe_find_text(driver, '#module_product_title_1 > div > div > h1')
        print("Product Title:", product_title)
        
        brand = safe_find_text(driver, '#module_product_brand_1 > div > a.pdp-link.pdp-link_size_s.pdp-link_theme_blue.pdp-product-brand__brand-link')
        print(brand)

        current_date = datetime.now().strftime("%Y-%m-%d")

        current_price = safe_find_text(driver, '#module_product_price_1 > div > div > div > span.notranslate.pdp-price.pdp-price_type_deleted.pdp-price_color_lightgray.pdp-price_size_xs')
        print(current_price)

        discount_price = safe_find_text(driver, '#module_product_price_1 > div > div > span')
        print(discount_price)   

        discount = safe_find_text(driver, '#module_product_price_1 > div > div > div > span.pdp-product-price__discount')
        print(discount)  

        rating_count = safe_find_text(driver, '#module_product_review_star_1 > div > a')
        print(rating_count)

        color = safe_find_text(driver, '#module_sku-select > div > div:nth-child(1) > div > div > div.sku-prop-content-header > span')
        print(color)

        capacity = safe_find_text(driver, '#module_sku-select > div > div:nth-child(2) > div > div > div.sku-prop-content-header > span')
        print(capacity)

        store = safe_find_text(driver, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[6]/div/div[1]/div[1]/div[2]/a[1]',By.XPATH)
        print(store)

        product_info = {
            'Platform': 'Shopee',
            'URL': url,
            'Date_Crawl': current_date,
            'product_name': product_title,
            'brand': brand,
            'current_price': current_price,
            'discount_price': discount_price,
            'discount': discount,
            'rating_count': rating_count,
            'color': color,
            'capacity': capacity,
            'store': store,
        }
        
        products.append(product_info)
    except TimeoutException:
        print("Timed out waiting for page to load")
    finally:
        return products

def safe_find_text(driver, selector, by=By.CSS_SELECTOR):
    try:
        element = driver.find_element(by, selector)
        return element.text
    except NoSuchElementException:
        return None

In [18]:
# Khởi tạo undetected_chromedriver với các tuỳ chỉnh
options = uc.ChromeOptions()
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
driver.implicitly_wait(10)

products = []

for link in links:
    products.extend(get_product_info(driver, link))

driver.quit()  # Đóng driver sau khi hoàn thành tất cả các lần gọi hàm

# Lưu kết quả vào DataFrame và xuất ra CSV
product_info_df = pd.DataFrame(products)
product_info_df.to_csv('product_smartphone_Lazada.csv', mode='a', header=not os.path.exists('product_smartphone_Lazada.csv'), index=False, encoding='utf-8-sig')
print(product_info_df)

https://www.lazada.vn/products/iphone-15-hang-chinh-hang-vna-i2417259469.html
Product Title: iPhone 15 - Hàng Chính Hãng VN/A
Apple
25.990.000 ₫
24.690.000 ₫
-5%
147 đánh giá
Black
256GB
Apple Flagship Store
https://www.lazada.vn/products/iphone-15-plus-hang-chinh-hang-vna-i2417254495.html
Product Title: iPhone 15 Plus - Hàng Chính Hãng VN/A
Apple
34.990.000 ₫
32.990.000 ₫
-6%
91 đánh giá
Yellow
512GB
Apple Flagship Store
https://www.lazada.vn/products/apple-iphone-15-128gb-chinh-hang-vna-i2417026282.html
Product Title: Apple iPhone 15 128GB Chính hãng (VN/A)
Apple
24.990.000 ₫
23.380.000 ₫
-6%
22 đánh giá
Đen
None
Di Động Việt Store - AAR
https://www.lazada.vn/products/iphone-15-pro-max-hang-chinh-hang-vna-i2417233670.html
Product Title: iPhone 15 Pro Max - Hàng Chính Hãng VN/A
Apple
34.999.000 ₫
31.990.000 ₫
-9%
1748 đánh giá
Natural Titanium
256GB
Apple Flagship Store
https://www.lazada.vn/products/iphone-15-pro-hang-chinh-hang-vna-i2417249565.html
Product Title: iPhone 15 Pro - Hàn

## Sau khi crawl xong sẽ copy cột sold_quantity ở bên product_link_Smartphone.csv và đưa vào product_smarphone_Lazada.csv

## Get comment

In [22]:
# Khởi tạo undetected_chromedriver với các tuỳ chỉnh
options = uc.ChromeOptions()
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)

# Dùng vòng lặp để lấy comment từ mỗi link và thêm vào DataFrame chính
df_list = []

for link in links:
    driver.get(link)
    count = 1
    try:
        # Chờ cho đến khi phần tử hiện diện, bạn có thể thêm WebDriverWait ở đây nếu cần
        selector = "#module_product_review > div > div"
        element = driver.find_element(By.CSS_SELECTOR, selector)
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
    except NoSuchElementException:
        print(f"Couldn't find the reviews section for {link}")
        continue
    while True:
        try:
            print("Crawl Comment Page " + str(count))
            # Giả sử mỗi page có n comment
            for j in range(1, 6):  # Giả định mỗi trang có 5 comments để kiểm tra
                # Thu thập tất cả thông tin cho mỗi comment
                try:
                    # Dùng Xpath cho time_comment
                    elem_time = driver.find_element(By.XPATH, f"/html/body/div[4]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[1]/div[{j}]/div[1]/span")
                    time_comment = elem_time.text

                    # Lấy tên người comment
                    elem_name = driver.find_element(By.CSS_SELECTOR, f"#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div:nth-child({j}) > div.middle > span:nth-child(1)")  # Cập nhật selector cho phù hợp
                    name_comment = elem_name.text
                                                                        
                    # Lấy nội dung comment
                    elem_content = driver.find_element(By.CSS_SELECTOR, f"#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div:nth-child({j}) > div.item-content > div.content")  # Cập nhật selector cho phù hợp
                    content_comment = elem_content.text

                    # Lấy thông tin sản phẩm từ comment
                    elem_info = driver.find_element(By.CSS_SELECTOR, f"#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div:nth-child({j}) > div.item-content > div.skuInfo")  # Cập nhật selector cho phù hợp
                    info_comment = elem_info.text

                    # Lấy số lượt thích của comment
                    elem_like = driver.find_element(By.CSS_SELECTOR, f"#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div:nth-child({j}) > div.item-content > div.bottom > span.left > span > span")  # Cập nhật selector cho phù hợp
                    like_comment = elem_like.text

                    # Tạo một dòng DataFrame từ thông tin đã thu thập
                    df = pd.DataFrame([[link, time_comment, name_comment, content_comment, info_comment, like_comment]],
                                      columns=['link_item', 'time_comment', 'name_cmt', 'content_cmt', 'info_cmt', 'like_count'])
                    df_list.append(df)
                except Exception as e:
                    print(f"Error crawling comment {j} on page {count}: {e}")
                    break

            try:
                # Kiểm tra sự tồn tại của nút "Trang tiếp theo"
                next_page_btn = driver.find_element(By.XPATH, "/html/body/div[4]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[2]/div/button[2]")
                # Nếu thuộc tính "disabled" tồn tại, nghĩa là không còn trang nào khác để điều hướng
                if next_page_btn.get_attribute("disabled") is not None:
                    print("Reached the last page. Exiting loop.")
                    break
                else:
                    # Nếu nút không bị vô hiệu hóa, click vào nút để đi đến trang tiếp theo
                    next_page_btn.click()
                    print("Clicked on the next page button!")
                    sleep(random.randint(1, 3))
                    count += 1
            except NoSuchElementException:
                # Khi không tìm thấy nút, có nghĩa là có lỗi xảy ra hoặc cấu trúc trang đã thay đổi
                print("Button not found. It's possible that the page structure has changed.")
                break
        except Exception as e:
            # Log lỗi không mong muốn khác
            print(f"An error occurred: {e}")
            break

# Kết hợp tất cả các DataFrame trong danh sách thành một DataFrame lớn
final_df = pd.concat(df_list, ignore_index=True)

Crawl Comment Page 1
Clicked on the next page button!
Crawl Comment Page 2
Clicked on the next page button!
Crawl Comment Page 3
Error crawling comment 4 on page 3: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=123.0.6312.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00514CE3+225091]
	(No symbol) [0x00444E31]
	(No symbol) [0x002E9A7A]
	(No symbol) [0x002EDEB0]
	(No symbol) [0x002EF862]
	(No symbol) [0x002EF8E0]
	(No symbol) [0x0031DC71]
	(No symbol) [0x0033F55C]
	(No symbol) [0x0031930E]
	(No symbol) [0x0033F7F4]
	(No symbol) [0x00355CB0]
	(No symbol) [0x0033F2F6]
	(No symbol) [0x003179B9]
	(No symbol) [0x0031879D]
	sqlite3_dbdata_init [0x00989A83+4064547]
	sqlite3_dbdata_init [0x0099108A+4094762]
	sqlite3_dbdata_init [0x0098B988+4072488]
	sqlite3_dbdata_init [0x0068C9E9+930953]
	(No sy

In [39]:
final_df

,link_item,time_comment,name_cmt,content_cmt,info_cmt,like_count
0,https://www.lazada.vn/products/iphone-15-hang-...,14 thg 11 2023,T***.,Mua hàng chính hãng tại Apple flagship store t...,"Nhóm Màu:Pink, Dung Lượng Lưu Trữ:128GB",4
1,https://www.lazada.vn/products/iphone-15-hang-...,02 thg 12 2023,B***h,Chất lượng: \n- Giao hàng nhanh. \n- SP đúng ...,"Nhóm Màu:Xanh dương, Dung Lượng Lưu Trữ:128GB",0
2,https://www.lazada.vn/products/iphone-15-hang-...,17 giờ trước,TRAN T.,"Shop chuẩn bị giao hàng nhanh, hình ảnh mang t...","Nhóm Màu:Black, Dung Lượng Lưu Trữ:128GB",0
3,https://www.lazada.vn/products/iphone-15-hang-...,17 thg 10 2023,T***.,"Giao hàng đúng hẹn, chất lượng máy tốt","Nhóm Màu:Màu Xanh Lá Cây, Dung Lượng Lưu Trữ:2...",0
4,https://www.lazada.vn/products/iphone-15-hang-...,30 thg 9 2023,Trang C.,"Màu hồng cưng xiu, triệu like","Nhóm Màu:Hồng, Dung Lượng Lưu Trữ:256GB",6
...,...,...,...,...,...,...
1577,https://www.lazada.vn/products/dien-thoai-sams...,21 thg 7 2023,Phung P.,,"Nhóm Màu:Đen, Dung Lượng Lưu Trữ:8GB/256GB",0
1578,https://www.lazada.vn/products/dien-thoai-sams...,30 thg 7 2023,Ngô M.,,"Nhóm Màu:Đen, Dung Lượng Lưu Trữ:8GB/256GB",0
1579,https://www.lazada.vn/products/dien-thoai-sams...,27 thg 7 2023,Nguyễn Q.,,"Nhóm Màu:Kem, Dung Lượng Lưu Trữ:8GB/256GB",0
1580,https://www.lazada.vn/products/dien-thoai-sams...,19 thg 7 2023,Q***.,chất lượng ok NHƯNG mình đề Góp Ý Nk Là khi kh...,"Nhóm Màu:Xanh Lục, Dung Lượng Lưu Trữ:8GB/256GB",0


In [40]:
final_df.to_csv('comment_Smarphone.csv', mode='a', header=not os.path.exists('comment_Smarphone.csv'), index=False, encoding='utf-8-sig')

In [41]:
df1 = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang TMĐT\Lazada\product_smartphone_Lazada.csv")
df2 = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang TMĐT\Lazada\comment_Smarphone.csv")
merge_df = pd.merge(df1, df2, on='URL', how='inner')
merge_df

,Platform,URL,Date_Crawl,product_name,brand,current_price,discount_price,discount,rating_count,color,capacity,store,sold_quantity,time_comment,name_cmt,content_cmt,info_cmt,like_count
0,Shopee,https://www.lazada.vn/products/iphone-15-hang-...,3/13/2024,iPhone 15 - Hàng Chính Hãng VN/A,Apple,25.990.000 ₫,24.690.000 ₫,-5%,147 đánh giá,Black,256GB,Apple Flagship Store,707 Đã bán,14 thg 11 2023,T***.,Mua hàng chính hãng tại Apple flagship store t...,"Nhóm Màu:Pink, Dung Lượng Lưu Trữ:128GB",4
1,Shopee,https://www.lazada.vn/products/iphone-15-hang-...,3/13/2024,iPhone 15 - Hàng Chính Hãng VN/A,Apple,25.990.000 ₫,24.690.000 ₫,-5%,147 đánh giá,Black,256GB,Apple Flagship Store,707 Đã bán,02 thg 12 2023,B***h,Chất lượng: \n- Giao hàng nhanh. \n- SP đúng ...,"Nhóm Màu:Xanh dương, Dung Lượng Lưu Trữ:128GB",0
2,Shopee,https://www.lazada.vn/products/iphone-15-hang-...,3/13/2024,iPhone 15 - Hàng Chính Hãng VN/A,Apple,25.990.000 ₫,24.690.000 ₫,-5%,147 đánh giá,Black,256GB,Apple Flagship Store,707 Đã bán,17 giờ trước,TRAN T.,"Shop chuẩn bị giao hàng nhanh, hình ảnh mang t...","Nhóm Màu:Black, Dung Lượng Lưu Trữ:128GB",0
3,Shopee,https://www.lazada.vn/products/iphone-15-hang-...,3/13/2024,iPhone 15 - Hàng Chính Hãng VN/A,Apple,25.990.000 ₫,24.690.000 ₫,-5%,147 đánh giá,Black,256GB,Apple Flagship Store,707 Đã bán,17 thg 10 2023,T***.,"Giao hàng đúng hẹn, chất lượng máy tốt","Nhóm Màu:Màu Xanh Lá Cây, Dung Lượng Lưu Trữ:2...",0
4,Shopee,https://www.lazada.vn/products/iphone-15-hang-...,3/13/2024,iPhone 15 - Hàng Chính Hãng VN/A,Apple,25.990.000 ₫,24.690.000 ₫,-5%,147 đánh giá,Black,256GB,Apple Flagship Store,707 Đã bán,30 thg 9 2023,Trang C.,"Màu hồng cưng xiu, triệu like","Nhóm Màu:Hồng, Dung Lượng Lưu Trữ:256GB",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577,Shopee,https://www.lazada.vn/products/dien-thoai-sams...,3/13/2024,Điện thoại Samsung Galaxy S23 Ultra,Samsung,31.990.000 ₫,27.190.000 ₫,-15%,43 đánh giá,Tím,8GB/256GB,NGUYỄN KIM,201,21 thg 7 2023,Phung P.,NaN,"Nhóm Màu:Đen, Dung Lượng Lưu Trữ:8GB/256GB",0
1578,Shopee,https://www.lazada.vn/products/dien-thoai-sams...,3/13/2024,Điện thoại Samsung Galaxy S23 Ultra,Samsung,31.990.000 ₫,27.190.000 ₫,-15%,43 đánh giá,Tím,8GB/256GB,NGUYỄN KIM,201,30 thg 7 2023,Ngô M.,NaN,"Nhóm Màu:Đen, Dung Lượng Lưu Trữ:8GB/256GB",0
1579,Shopee,https://www.lazada.vn/products/dien-thoai-sams...,3/13/2024,Điện thoại Samsung Galaxy S23 Ultra,Samsung,31.990.000 ₫,27.190.000 ₫,-15%,43 đánh giá,Tím,8GB/256GB,NGUYỄN KIM,201,27 thg 7 2023,Nguyễn Q.,NaN,"Nhóm Màu:Kem, Dung Lượng Lưu Trữ:8GB/256GB",0
1580,Shopee,https://www.lazada.vn/products/dien-thoai-sams...,3/13/2024,Điện thoại Samsung Galaxy S23 Ultra,Samsung,31.990.000 ₫,27.190.000 ₫,-15%,43 đánh giá,Tím,8GB/256GB,NGUYỄN KIM,201,19 thg 7 2023,Q***.,chất lượng ok NHƯNG mình đề Góp Ý Nk Là khi kh...,"Nhóm Màu:Xanh Lục, Dung Lượng Lưu Trữ:8GB/256GB",0


In [42]:
merge_df.to_csv('Smarphone_Lazada.csv', mode='a', header=not os.path.exists('Smarphone_Lazada.csv'), index=False, encoding='utf-8-sig')